In [1]:
import numpy as np
import pandas as pd
from os import getcwd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
import os
import pickle

In [2]:
def set_path(): 
   abspath = getcwd()
   dname = os.path.dirname(abspath)
   os.chdir(dname)

In [3]:
# Vetorize content (abstract + title) by *authorId*

def vectorize_content(df, authId):
   df_auth = df['authorId']== authId
   df_content = df_auth['abstract'].tolist() + df_auth['title'].tolist()
   
   #vectorize literature style
   auth_NLP = CountVectorizer(input = 'content')
   auth_NLP.fit_transform(df_content)
   return auth_NLP

# get estimates from categorical variables (year and venue) for each author by random forest 
def estimateCat(df, authId, estim = 20):
   cats = df[['year', 'venues_le']]
   target = df['authorId'] == authId
   clf = RandomForestClassifier(n_estimators= estim)
   clf = clf.fit(cats, target)
   
   return clf #prob of pair of cats by author

def author_enc(df, authId):
   nlp = vectorize_content(df, authId)
   cat = estimateCat(df, authId)

   merged = [nlp,cat]
   merged[0].get_feature_names_out()

   #write authorId encoded into file:
   with open(f"models/{authId}.pkl", 'wb') as f:
      pickle.dump(file = f, obj =merged)

def encoding_all():
   set_path()
   df = pd.read_pickle('data/processed/clean_df.pkl')
   authIds = df['authorId'].unique()
   for authId in authIds:
      author_enc(df, authId)
   
encoding_all()



